# Self Refine

Self-Refine 的核心是增加了跟 LLM 的交互次数

In [6]:
from utils import ask

## 基于LLM 进行翻译

In [3]:
# 需要翻译的原文
source_text = """
Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they should be applied to AI applications, not to general-purpose AI technology.
"""

In [4]:
# 构造第一次跟 LLM 交互的 system prompt 和 user content

## system prompt
first_translate_system_message = f"You are an expert linguist, specializing in translation from English to Traditional Chinese."

## user prompt

first_user_translation_prompt = f"""This is an English to Traditional Chinese translation, please provide the Traditional Chinese translation for this text. \\
Do not provide any explanations or text apart from the translation.
English: {source_text}

Traditional Chinese:"""

## 将 source text 作为参数传入，构造好完整的输入
first_user_prompt = first_user_translation_prompt.format(source_text=source_text)

print(first_user_prompt)

This is an English to Traditional Chinese translation, please provide the Traditional Chinese translation for this text. \
Do not provide any explanations or text apart from the translation.
English: 
Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they should be applied to AI applications, not to general-purpose AI technology.


Traditional Chinese:


In [7]:
messages = [
    {"role":"system", "content": first_translate_system_message},
    {"role":"user", "content": first_user_prompt}
]

first_response = ask(messages=messages)

translation_1 = first_response.choices[0].message.content
print(translation_1)

上週，我在美國國會大廈的一場活動中，就人工智慧與監管議題發表演講，現場有立法及商業領袖參與。開源社群在抵禦可能扼殺創新的監管措施方面取得的進展令我振奮。但開源的反對者正不斷調整論點，最新的擔憂集中在開源對國家安全的影響。我希望我們能繼續守護開源精神！

根據我與立法者的交流，美國聯邦政府對人工智慧風險的現實認知有所進展，這點令人鼓舞。需要明確的是，防護措施是必要的，但它們應針對人工智慧的具體應用，而非通用型人工智慧技術本身。


## 基于LLM 进行评价

第一次交互我们给 LLM 设定为 翻译家的角色，进行翻译任务。但是第二次交互的目的不再是翻译，而是给出评价。我们还是需要构造好输入的提示词。

In [10]:
reflect_system_prompt = """You are an expert linguist specializing in translation from English to Traditional Chinese.
You will be provided with a source text and its translation and your goal is to improve the translation."""

In [11]:
reflect_prompt = f"""Your task is to carefully read a source text and a translation from English to Traditional Chinese, and then give constructive criticism and helpful suggestions to improve the translation. \\

The source text and initial translation, delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT> and <TRANSLATION></TRANSLATION>, are as follows:

<SOURCE_TEXT>
{source_text}
</SOURCE_TEXT>

<TRANSLATION>
{translation_1}
</TRANSLATION>

When writing suggestions, pay attention to whether there are ways to improve the translation's \\n\\
(i) accuracy (by correcting errors of addition, mistranslation, omission, or untranslated text),\\n\\
(ii) fluency (by applying Traditional Chinese grammar, spelling and punctuation rules, and ensuring there are no unnecessary repetitions),\\n\\
(iii) style (by ensuring the translations reflect the style of the source text and takes into account any cultural context),\\n\\
(iv) terminology (by ensuring terminology use is consistent and reflects the source text domain; and by only ensuring you use equivalent idioms Traditional Chinese).\\n\\

Write a list of specific, helpful and constructive suggestions for improving the translation.
Each suggestion should address one specific part of the translation.
Output only the suggestions and nothing else."""

feedback_user_prompt = reflect_prompt.format(
        source_text=source_text,
        translation_1=translation_1,
    )


In [12]:
message2 = [
    {"role":"system", "content": reflect_system_prompt},
    {"role":"user", "content": feedback_user_prompt}
]

response = ask(messages=message2)

reflection = response.choices[0].message.content
print(reflection)

1. 建議將「美國國會大廈」改為「美國國會山莊」，以更貼近臺灣常用譯名，並符合「Capitol」在政治語境中的慣用表述。
2. 「就人工智慧與監管議題發表演講」可調整為「針對人工智慧與監管議題發表演說」，使動詞「spoke about」的翻譯更符合正式演講場合的語體。
3. 「開源社群在抵禦可能扼殺創新的監管措施方面取得的進展令我振奮」中「振奮」一詞建議改為「感到鼓舞」，以更準確對應原文「I’m encouraged」的語意與語氣。
4. 「開源的反對者」可考慮譯為「開源的反對陣營」或「反對開源的一方」，使「opponents」的譯法更具團體指涉性。
5. 「最新的擔憂集中在開源對國家安全的影響」建議微調為「最新的憂慮聚焦於開源對國家安全的影響」，使「centering on」的動態感更突出。
6. 「守護開源精神」中的「精神」為譯者添加，建議刪除以緊扣原文「protecting open source」的具體指涉，改為「守護開源模式／生態」。
7. 「現實認知有所進展」可優化為「已能更務實地掌握其風險」，使「getting a realistic grasp」的動態過程更清晰，並補足原文「AI’s risks」中的「其」指代。
8. 「防護措施」建議改為「防護機制」或「安全防護」，以更貼近技術監管領域中「guardrails」的比喻意涵。
9. 「針對人工智慧的具體應用，而非通用型人工智慧技術本身」中「具體應用」與「技術本身」的對比可加強，建議改為「應適用於人工智慧的具體應用層面，而非通用型人工智慧技術本身」，使邏輯層次更分明。


## 基于LLM的二次翻译

第三次交互，大家可以想到就是将第一次翻译和第二次反馈的结果一起丢给 LLM，然后让 LLM继续 进行二次翻译，并得到优化的结果。

In [13]:
# system prompt
refiner_system_prompt = f"You are an expert linguist, specializing in translation editing from English to Traditional Chinese."

# user content
refined_translate_user_prompt = f"""Your task is to carefully read, then edit, a translation from English to Traditional Chinese, taking into
account a list of expert suggestions and constructive criticisms.

The source text, the initial translation, and the expert linguist suggestions are delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT>, <TRANSLATION></TRANSLATION> and <EXPERT_SUGGESTIONS></EXPERT_SUGGESTIONS> \\
as follows:

<SOURCE_TEXT>
{source_text}
</SOURCE_TEXT>

<TRANSLATION>
{translation_1}
</TRANSLATION>

<EXPERT_SUGGESTIONS>
{reflection}
</EXPERT_SUGGESTIONS>

Please take into account the expert suggestions when editing the translation. Edit the translation by ensuring:

(i) accuracy (by correcting errors of addition, mistranslation, omission, or untranslated text),
(ii) fluency (by applying Traditional Chinese grammar, spelling and punctuation rules and ensuring there are no unnecessary repetitions), \\
(iii) style (by ensuring the translations reflect the style of the source text)
(iv) terminology (inappropriate for context, inconsistent use), or
(v) other errors.

Output only the new translation and nothing else."""

refined_user_prompt = refined_translate_user_prompt.format(
        source_text=source_text,
        translation_1=translation_1,
        reflection=reflection,
    )

message3 = [
    {"role":"system", "content": refiner_system_prompt},
    {"role":"user", "content": refined_user_prompt}
]

response = ask(messages=message3)

translation_2 = response.choices[0].message.content
print(translation_2)

上週，我在美國國會山莊的一場活動中，針對人工智慧與監管議題發表演說，現場有立法及商業領袖參與。開源社群在抵禦可能扼殺創新的監管措施方面取得的進展，令我感到鼓舞。但反對開源的一方正不斷調整論點，最新的憂慮聚焦於開源對國家安全的影響。我希望我們能持續守護開源生態！

根據我與立法者的交流，美國聯邦政府已能更務實地掌握人工智慧的風險，這點令人鼓舞。需要明確的是，安全防護機制確有必要，但它們應適用於人工智慧的具體應用層面，而非通用型人工智慧技術本身。


## 总结

1. Self refine 的核心其实就是跟 LLM 的多次交互，每次交互让 LLM 扮演不同的角色执行不同的任务从而得到最终的结果
2. 由此我们可以推论出一个暂时性的结论，跟 LLM 的交互实际上就是控制好输入的提示词，那么怎么调用外部的工具呢